In [251]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import math
import pickle
import torch
from tqdm import tqdm, tqdm_notebook
from sklearn.externals import joblib

from sklearn.metrics import balanced_accuracy_score, roc_auc_score, \
                            classification_report, log_loss

from sklearn.linear_model import LogisticRegression

from multi_agent_simulator import BiddingAgent, BiddingEnvironment, BidStrategy

import matplotlib as mpl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks")
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
validation = pd.read_hdf('preprocessed.h5', 'validation')

# pCTR using different models

In [212]:
# load the trained LR
lr = pickle.load(open( "lr_model", "rb" ))

# user LR to predict pCTR
pCTR_lr = lr.predict_proba(validation.drop(columns=['payprice', 'click']))[:, 1]

In [253]:
# load the best NN
file = 'nmodel_roc_0.856_balanced_acc_0.000_model_loss_0.017.pt'
PATH = '/content/gdrive/My Drive/Colab Notebooks/' + file

model = torch.load(PATH)
model.eval()

# use NN to predict pCTR
batches = np.array_split(validation, 200)

pCTR_NN = []
for batch in batches:
    data = batch.drop(columns=['click', 'payprice']).values
    target = batch['click'].values
    
    data = torch.from_numpy(data).float()
    target = torch.from_numpy(target).float()
    
    output = model(data)
    
    prediction = (output.data).float()
    y_hat_prob = prediction.cpu().numpy().squeeze()
    
    pCTR_NN.extend(y_hat_prob)
    
pCTR_NN = np.array(pCTR_NN)

FileNotFoundError: [Errno 2] No such file or directory: 'nmodel_roc_0.856_balanced_acc_0.000_model_loss_0.017.pt'

In [210]:
# Load different models
xgb = joblib.load('xgb_model')
pCTR_XGB = xgb.predict(validation.drop(columns=['payprice', 'click']).values)

/Users/davidvanrooij/.pyenv/versions/3.6.7/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [211]:
# get baseline avgCTR
no_click, click = np.bincount(validation['click'].values)
avgCTR = click / (no_click + click)

# Set environment & agents

In [ ]:
# constant bidding
environment = BiddingEnvironment(validation.copy())

In [7]:
# From previous analysis the best performing models are included

bids_const1 = BidStrategy.const_bidding(300, environment.length)
bids_const2 = BidStrategy.const_bidding(1000, environment.length)

bids_random1 = BidStrategy.random_bidding(300, 400, environment.length)
bids_random2 = BidStrategy.random_bidding(1000, 1500, environment.length)
bids_random3 = BidStrategy.random_bidding(10000, 15000, environment.length)
bids_random4 = BidStrategy.random_bidding(100000, 150000, environment.length)


bids_opportunistic_lr_1 = BidStrategy.opportunistic(pCTR_lr, 150000, 0.8)
bids_opportunistic_lr_2 = BidStrategy.opportunistic(pCTR_lr, 200000, 0.9)

bids_opportunistic_NN_1 = BidStrategy.opportunistic(pCTR_NN, 150000, 0.8)
bids_opportunistic_NN_2 = BidStrategy.opportunistic(pCTR_NN, 200000, 0.9)

bids_opportunistic_XGB_1 = BidStrategy.opportunistic(pCTR_XGB, 150000, 0.8)
bids_opportunistic_XGB_2 = BidStrategy.opportunistic(pCTR_XGB, 200000, 0.9)


bids_linear_lr_1 = BidStrategy.linear_bidding(pCTR_lr, avgCTR, 3.52)
bids_linear_lr_2 = BidStrategy.linear_bidding(pCTR_lr, avgCTR, 3.54)

bids_linear_NN_1 = BidStrategy.linear_bidding(pCTR_NN, avgCTR, 70)
bids_linear_NN_2 = BidStrategy.linear_bidding(pCTR_NN, avgCTR, 60)

bids_linear_XGB_1 = BidStrategy.linear_bidding(pCTR_XGB, )
bids_linear_XGB_2 = BidStrategy.linear_bidding(pCTR_XGB, )


bids_ortb1_lr_1 = BidStrategy.ortb1(pCTR_lr, 15, 0.000025)
bids_ortb1_lr_2 = BidStrategy.ortb1(pCTR_lr, 45, 0.000047)

bids_ortb1_NN_1 = BidStrategy.ortb1(pCTR_NN, 30 ,0.000002)
bids_ortb1_NN_2 = BidStrategy.ortb1(pCTR_NN, 50 ,0.000002)

bids_ortb1_XGB_1 = BidStrategy.ortb1(pCTR_XGB, 10, 0.000089)
bids_ortb1_XGB_2 = BidStrategy.ortb1(pCTR_XGB, 10, 0.000084)


bids_ortb2_lr_1 = BidStrategy.ortb2(pCTR_lr, 90, 0.000090)
bids_ortb2_lr_2 = BidStrategy.ortb2(pCTR_lr, 80, 0.000086)

bids_ortb2_NN_1 = BidStrategy.ortb2(pCTR_NN, 20, 6.210526e-7)
bids_ortb2_NN_2 = BidStrategy.ortb2(pCTR_NN, 50, 2.184211e-6)

bids_ortb2_XGB_1 = BidStrategy.ortb2(pCTR_XGB, 20, 0.000047)
bids_ortb2_XGB_2 = BidStrategy.ortb2(pCTR_XGB, 20, 0.000100)


bids_optimal_second_price_lr_1 = BidStrategy.second_price(pCTR_lr, )
bids_optimal_second_price_lr_2 = BidStrategy.second_price(pCTR_lr, )

bids_optimal_second_price_NN_1 = BidStrategy.second_price(pCTR_NN, 6250*1000, 303925,)
bids_optimal_second_price_NN_2 = BidStrategy.second_price(pCTR_NN, 6250*1000, 303925,)

bids_optimal_second_price_XGB_1 = BidStrategy.second_price(pCTR_XGB, 6250*1000, 303925, 240)
bids_optimal_second_price_XGB_2 = BidStrategy.second_price(pCTR_XGB, 6250*1000, 303925, 230)


# register bids to the environment
environment.register_bid(bids_const1)
environment.register_bid(bids_const2)
environment.register_bid(bids_random1)
environment.register_bid(bids_random2)
environment.register_bid(bids_random3)
environment.register_bid(bids_random4)
environment.register_bid(bids_opportunistic1)
environment.register_bid(bids_opportunistic2)
environment.register_bid(bids_opportunistic1)
environment.register_bid(bids_opportunistic2)
environment.register_bid(bids_opportunistic1)
environment.register_bid(bids_opportunistic2)
environment.register_bid(bids_linear_lr_1)
environment.register_bid(bids_linear_lr_2)
environment.register_bid(bids_linear_NN_1)
environment.register_bid(bids_linear_NN_2)
environment.register_bid(bids_linear_XGB_1)
environment.register_bid(bids_linear_XGB_2)
environment.register_bid(bids_ortb1_lr_1)
environment.register_bid(bids_ortb1_lr_2)
environment.register_bid(bids_ortb1_NN_1)
environment.register_bid(bids_ortb1_NN_2)
environment.register_bid(bids_ortb1_XGB_1)
environment.register_bid(bids_ortb1_XGB_2)
environment.register_bid(bids_ortb2_lr_1)
environment.register_bid(bids_ortb2_lr_2)
environment.register_bid(bids_ortb2_NN_1)
environment.register_bid(bids_ortb2_NN_2)
environment.register_bid(bids_ortb2_XGB_1)
environment.register_bid(bids_ortb2_XGB_2)
environment.register_bid(bids_optimal_second_price_lr_1)
environment.register_bid(bids_optimal_second_price_lr_2)
environment.register_bid(bids_optimal_second_price_NN_1)
environment.register_bid(bids_optimal_second_price_NN_2)
environment.register_bid(bids_optimal_second_price_XGB_1)
environment.register_bid(bids_optimal_second_price_XGB_2)

print('Registerd {} agent\' bids to the environment'.format(environment.number_bids))